# Тонкая настройка моделей Open AI

Этот блокнот основан на текущих рекомендациях, представленных в документации [Тонкая настройка](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) от Open AI.

Тонкая настройка улучшает производительность базовых моделей для вашего приложения путем дообучения с использованием дополнительных данных и контекста, относящихся к конкретному случаю использования или сценарию. Обратите внимание, что методы проектирования подсказок, такие как _few shot learning_ и _retrieval augmented generation_, позволяют улучшить стандартную подсказку с помощью релевантных данных для повышения качества. Однако эти подходы ограничены максимальным размером окна токенов целевой базовой модели.

С помощью тонкой настройки мы фактически дообучаем саму модель с необходимыми данными (что позволяет использовать гораздо больше примеров, чем помещается в максимальное окно токенов) — и разворачиваем _кастомизированную_ версию модели, которой больше не нужно предоставлять примеры во время вывода. Это не только повышает эффективность нашего дизайна подсказок (у нас появляется больше гибкости в использовании окна токенов для других задач), но и потенциально снижает наши затраты (за счет уменьшения количества токенов, которые нужно отправлять модели во время вывода).

Тонкая настройка включает 4 шага:
1. Подготовить обучающие данные и загрузить их.
1. Запустить задачу обучения, чтобы получить тонко настроенную модель.
1. Оценить тонко настроенную модель и при необходимости повторить для улучшения качества.
1. Развернуть тонко настроенную модель для вывода, когда результат удовлетворит.

Обратите внимание, что не все базовые модели поддерживают тонкую настройку — [проверьте документацию OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) для получения актуальной информации. Вы также можете тонко настроить ранее тонко настроенную модель. В этом руководстве мы будем использовать `gpt-35-turbo` в качестве целевой базовой модели для тонкой настройки.

---


### Шаг 1.1: Подготовьте ваш набор данных

Давайте создадим чат-бота, который поможет вам понять периодическую таблицу элементов, отвечая на вопросы об элементе с помощью лимерика. В _этом_ простом руководстве мы просто создадим набор данных для обучения модели с несколькими примерами ответов, показывающими ожидаемый формат данных. В реальном случае использования вам потребуется создать набор данных с гораздо большим количеством примеров. Вы также можете использовать открытый набор данных (для вашей области применения), если он существует, и преобразовать его для использования в дообучении.

Поскольку мы сосредоточены на `gpt-35-turbo` и ищем ответ в один ход (chat completion), мы можем создать примеры, используя [этот предложенный формат](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), отражающий требования OpenAI к chat completion. Если вы ожидаете многократный диалог, вы бы использовали [формат примеров для многократного диалога](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), который включает параметр `weight` для указания, какие сообщения должны использоваться (или нет) в процессе дообучения.

Для нашего руководства мы используем более простой формат с одним ходом. Данные представлены в формате [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) с одной записью на строку, каждая из которых представлена в виде объекта в формате JSON. Ниже приведён фрагмент с 2 записями в качестве примера — смотрите [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) для полного набора примеров (10 примеров), который мы используем для нашего руководства по дообучению. **Примечание:** Каждая запись _должна_ быть определена в одной строке (не разбита на несколько строк, как обычно в форматированном JSON-файле)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

В реальном случае использования вам потребуется гораздо больший набор примеров для хороших результатов — компромисс будет между качеством ответов и временем/стоимостью дообучения. Мы используем небольшой набор, чтобы быстро завершить дообучение и проиллюстрировать процесс. Смотрите [этот пример из OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) для более сложного руководства по дообучению.


---

### Шаг 1.2 Загрузите ваш набор данных

Загрузите данные с помощью Files API [как описано здесь](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Обратите внимание, что для запуска этого кода вы должны сначала выполнить следующие шаги:
 - Установить пакет `openai` для Python (убедитесь, что используете версию >=0.28.0 для последних функций)
 - Установить переменную окружения `OPENAI_API_KEY` со своим ключом API OpenAI
Чтобы узнать больше, смотрите [Руководство по настройке](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), предоставленное для курса.

Теперь запустите код, чтобы создать файл для загрузки из вашего локального файла JSONL.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Шаг 2.1: Создание задачи дообучения с помощью SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Шаг 2.2: Проверка статуса задания

Вот несколько действий, которые вы можете выполнить с помощью API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Список последних n заданий по дообучению
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Получить детали конкретного задания по дообучению
- `client.fine_tuning.jobs.cancel(<job_id>)` - Отменить задание по дообучению
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Список до n событий из задания
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Первым шагом процесса является _проверка файла обучения_, чтобы убедиться, что данные имеют правильный формат.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Шаг 2.3: Отслеживайте события для мониторинга прогресса


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Шаг 2.4: Просмотр статуса в панели управления OpenAI


Вы также можете просмотреть статус, посетив веб-сайт OpenAI и изучив раздел _Fine-tuning_ на платформе. Там будет показан статус текущей задачи, а также возможность отслеживать историю предыдущих запусков задач. На этом скриншоте видно, что предыдущий запуск завершился с ошибкой, а второй запуск прошёл успешно. Для контекста: это произошло, когда первый запуск использовал JSON-файл с неправильно отформатированными записями — после исправления второй запуск завершился успешно и модель стала доступна для использования.

![Fine-tuning job status](../../../../../translated_images/ru/fine-tuned-model-status.563271727bf7bfba.webp)


Вы также можете просмотреть сообщения о состоянии и метрики, прокрутив вниз в визуальной панели, как показано ниже:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/ru/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/ru/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### Шаг 3.1: Получение ID и тестирование дообученной модели в коде


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Шаг 3.2: Загрузка и тестирование дообученной модели в Playground

Теперь вы можете протестировать дообученную модель двумя способами. Во-первых, вы можете посетить Playground и использовать выпадающее меню Models, чтобы выбрать вашу недавно дообученную модель из списка доступных. Другой вариант — использовать опцию "Playground", показанную в панели Fine-tuning (см. скриншот выше), которая запускает следующий _сравнительный_ вид, показывающий версии базовой и дообученной модели бок о бок для быстрого оценивания.

![Fine-tuning job status](../../../../../translated_images/ru/fine-tuned-playground-compare.56e06f0ad8922016.webp)

Просто заполните системный контекст, использованный в ваших тренировочных данных, и введите ваш тестовый вопрос. Вы заметите, что обе стороны обновляются с идентичным контекстом и вопросом. Запустите сравнение, и вы увидите разницу в ответах между ними. _Обратите внимание, как дообученная модель формирует ответ в формате, который вы предоставили в своих примерах, в то время как базовая модель просто следует системному запросу_.

![Fine-tuning job status](../../../../../translated_images/ru/fine-tuned-playground-launch.5a26495c983c6350.webp)

Вы заметите, что сравнение также показывает количество токенов для каждой модели и время, затраченное на вывод. **Этот конкретный пример является упрощённым и предназначен для демонстрации процесса, но не отражает реальный набор данных или сценарий**. Вы можете заметить, что оба образца показывают одинаковое количество токенов (системный контекст и запрос пользователя идентичны), при этом дообученная модель тратит больше времени на вывод (кастомная модель).

В реальных сценариях вы не будете использовать такой игрушечный пример, а будете дообучать на реальных данных (например, каталог продуктов для службы поддержки клиентов), где качество ответа будет гораздо более заметным. В _таком_ контексте получение эквивалентного качества ответа с базовой моделью потребует более сложной настройки запросов, что увеличит использование токенов и, возможно, время обработки вывода. _Чтобы попробовать это, ознакомьтесь с примерами дообучения в OpenAI Cookbook для начала._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия по обеспечению точности, имейте в виду, что автоматический перевод может содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется использовать профессиональный перевод, выполненный человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
